##### Copyright 2022 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [1]:
#@title Copyright 2022 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/hub/tutorials/tf_hub_film_example"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/tf_hub_film_example.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/hub/blob/master/examples/colab/tf_hub_film_example.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/hub/examples/colab/tf_hub_film_example.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
  <td>
    <a href="/kaggle/input/film/tensorflow2/film/1"><img src="https://www.tensorflow.org/images/hub_logo_32px.png" />See TF Hub model</a>
  </td>
</table>

# Frame interpolation using the FILM model


Frame interpolation is the task of synthesizing many in-between images from a given set of images. The technique is often used for frame rate upsampling or creating slow-motion video effects.

In this colab, you will use the FILM model to do frame interpolation. The colab also provides code snippets to create videos from the interpolated in-between images.

For more information on FILM research, you can read more here:
- Google AI Blog: [Large Motion Frame Interpolation](https://ai.googleblog.com/2022/10/large-motion-frame-interpolation.html)
- Project Page: FILM: [Frame Interpolation for Large Motion](https://film-net.github.io/)


## Setup

In [2]:
!pip install mediapy
!sudo apt-get install -y ffmpeg

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)




ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).


0 upgraded, 0 newly installed, 0 to remove and 103 not upgraded.


In [3]:
import tensorflow as tf
import tensorflow_hub as hub

import requests
import numpy as np

from typing import Generator, Iterable, List, Optional
import mediapy as media

## Load the model from TFHub

To load a model from TensorFlow Hub you need the tfhub library and the model handle which is its documentation url.

In [4]:
model = hub.load("/kaggle/input/film/tensorflow2/film/1")

## Util function to load images from a url or locally

This function loads an image and make it ready to be used by the model later.

In [5]:
_UINT8_MAX_F = float(np.iinfo(np.uint8).max)

def load_image(img_url: str):
  """Returns an image with shape [height, width, num_channels], with pixels in [0..1] range, and type np.float32."""

  if (img_url.startswith("https")):
    user_agent = {'User-agent': 'Colab Sample (https://tensorflow.org)'}
    response = requests.get(img_url, headers=user_agent)
    image_data = response.content
  else:
    image_data = tf.io.read_file(img_url)

  image = tf.io.decode_image(image_data, channels=3)
  image_numpy = tf.cast(image, dtype=tf.float32).numpy()
  return image_numpy / _UINT8_MAX_F

In [6]:
# using images from the FILM repository (https://github.com/google-research/frame-interpolation/)

image_1_url = "https://github.com/google-research/frame-interpolation/blob/main/photos/one.png?raw=true"
image_2_url = "https://github.com/google-research/frame-interpolation/blob/main/photos/two.png?raw=true"

time = np.array([0.5], dtype=np.float32)

image1 = load_image(image_1_url)
image2 = load_image(image_2_url)

FILM's model input is a dictionary with the keys `time`, `x0`, `x1`:

- `time`: position of the interpolated frame. Midway is `0.5`.
- `x0`: is the initial frame.
- `x1`: is the final frame.

Both frames need to be normalized (done in the function `load_image` above) where each pixel is in the range of `[0..1]`.

`time` is a value between `[0..1]` and it says where the generated image should be. 0.5 is midway between the input images.

All three values need to have a batch dimension too.

In [7]:
image_1_url = "https://github.com/google-research/frame-interpolation/blob/main/photos/one.png?raw=true"
image_2_url = "https://github.com/google-research/frame-interpolation/blob/main/photos/two.png?raw=true"

time = np.array([0.5], dtype=np.float32)

image1 = load_image(image_1_url)
image2 = load_image(image_2_url)
input = {
    'time': np.expand_dims(time, axis=0), # adding the batch dimension to the time
     'x0': np.expand_dims(image1, axis=0), # adding the batch dimension to the image
     'x1': np.expand_dims(image2, axis=0)  # adding the batch dimension to the image
}
mid_frame = model(input)

In [8]:
def interpolate_frames(image1, image2):
    """Generate an interpolated frame between two images using FILM, with automatic resizing."""
    try:
        # Load and normalize images
        img1 = load_image(image1)
        img2 = load_image(image2)

        # If shapes differ, resize the second image to match the first
        if img1.shape != img2.shape:
            target_shape = img1.shape[:2]  # Use the shape (height, width) of the first image
            img2 = tf.image.resize(img2, target_shape).numpy()

        # Time input (interpolation at the middle point, i.e., 0.5)
        time = np.array([0.5], dtype=np.float32)

        # Prepare input as a dictionary
        input_data = {
            'time': np.expand_dims(time, axis=0),  # Add batch dimension to time
            'x0': np.expand_dims(img1, axis=0),   # Add batch dimension to image 1
            'x1': np.expand_dims(img2, axis=0)    # Add batch dimension to image 2
        }
        output=model(input_data)['image'][0].numpy()/1.05
        return output

    except Exception as e:
        return f"Error: {str(e)}"

        return output

    except Exception as e:
        return f"Error: {e}"

In [9]:
!pip install gradio

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


     ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.3/20.0 MB 9.1 MB/s eta 0:00:03

     ━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/20.0 MB 37.7 MB/s eta 0:00:01

     ━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/20.0 MB 76.0 MB/s eta 0:00:01

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺ 19.7/20.0 MB 171.1 MB/s eta 0:00:01

...

 done
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=5600 sha256=eb0ddc85b6a79a4fe51b3b0bdebbff0df9ba81a65b6f4bd9ad5a2e0f55d64902
  Stored in directory: /root/.cache/pip/wheels/81/c3/08/a4932aee56b934891932e4a1b189604f27d3d4e92b2ecd5ed4
Successfully built ffmpy


  Attempting uninstall: huggingface-hub


    Found existing installation: huggingface-hub 0.10.1
    Uninstalling huggingface-hub-0.10.1:


      Successfully uninstalled huggingface-hub-0.10.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cached-path 1.1.6 requires huggingface-hub<0.11.0,>=0.8.1, but you have huggingface-hub 0.16.4 which is incompatible.


In [10]:
import gradio as gr
image_input_1 = gr.inputs.Image(type="numpy", label="Upload or Paste URL for Image 1")
image_input_2 = gr.inputs.Image(type="numpy", label="Upload or Paste URL for Image 2")
output_image = gr.outputs.Image(type="numpy", label="Interpolated Frame")

# Customize with gradio UI options
ui = gr.Interface(
    fn=interpolate_frames,
    inputs=[image_input_1, image_input_2],
    outputs=output_image,
    title="Frame Interpolation",
    description="Upload two images or enter their URLs to generate an interpolated frame.",
    theme="compact"  # You can also choose 'huggingface', 'default', etc.
)

/opt/conda/lib/python3.7/site-packages/gradio/inputs.py:260: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  "Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components",
/opt/conda/lib/python3.7/site-packages/gradio/inputs.py:270: UserWarning: `optional` parameter is deprecated, and it has no effect
  optional=optional,
/opt/conda/lib/python3.7/site-packages/gradio/outputs.py:44: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  "Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components",
/opt/conda/lib/python3.7/site-packages/gradio/blocks.py:679: UserWarning: Cannot load compact. Caught Exception: The space compact does not exist
  warnings.warn(

IMPORTANT: You are using gradio version 3.34.0, however version 4.44.1 is available, please upgrade.
--------


In [ ]:
ui = gr.Interface(
    fn=interpolate_frames,
    inputs=[image_input_1, image_input_2],
    outputs=output_image,
    title="Frame Interpolation",
    description="Upload two images or enter their URLs to generate an interpolated frame.",
    theme='compact'# You can also choose 'huggingface', 'default', etc.
)

# Launch the app
ui.launch(debug=True, share=True)

The model outputs a couple of results but what you'll use here is the `image` key, whose value is the interpolated frame.

In [ ]:
print(mid_frame.keys())

In [ ]:
mid_frame['image'][0].numpy().shape

In [ ]:
print(mid_frame['image'].dtype)  # Check if it's float32 or uint8
print(mid_frame['image'].numpy().min(), mid_frame['image'].numpy().max())  # Check the value range

In [ ]:
frames = [image1, mid_frame['image'][0].numpy(), image2]

media.show_images(frames, titles=['input image one', 'generated image', 'input image two'], height=250)

Let's create a video from the generated frames

In [ ]:
media.show_video(frames, fps=3, title='FILM interpolated video')

## Define a Frame Interpolator Library

As you can see, the transition is not too smooth. 

To improve that you'll need many more interpolated frames.

You could just keep running the model many times with intermediary images but there is a better solution.

To generate many interpolated images and have a  smoother video you'll create an interpolator library.

In [ ]:
"""A wrapper class for running a frame interpolation based on the FILM model on TFHub

Usage:
  interpolator = Interpolator()
  result_batch = interpolator(image_batch_0, image_batch_1, batch_dt)
  Where image_batch_1 and image_batch_2 are numpy tensors with TF standard
  (B,H,W,C) layout, batch_dt is the sub-frame time in range [0..1], (B,) layout.
"""


def _pad_to_align(x, align):
  """Pads image batch x so width and height divide by align.

  Args:
    x: Image batch to align.
    align: Number to align to.

  Returns:
    1) An image padded so width % align == 0 and height % align == 0.
    2) A bounding box that can be fed readily to tf.image.crop_to_bounding_box
      to undo the padding.
  """
  # Input checking.
  assert np.ndim(x) == 4
  assert align > 0, 'align must be a positive number.'

  height, width = x.shape[-3:-1]
  height_to_pad = (align - height % align) if height % align != 0 else 0
  width_to_pad = (align - width % align) if width % align != 0 else 0

  bbox_to_pad = {
      'offset_height': height_to_pad // 2,
      'offset_width': width_to_pad // 2,
      'target_height': height + height_to_pad,
      'target_width': width + width_to_pad
  }
  padded_x = tf.image.pad_to_bounding_box(x, **bbox_to_pad)
  bbox_to_crop = {
      'offset_height': height_to_pad // 2,
      'offset_width': width_to_pad // 2,
      'target_height': height,
      'target_width': width
  }
  return padded_x, bbox_to_crop


class Interpolator:
  """A class for generating interpolated frames between two input frames.

  Uses the Film model from TFHub
  """

  def __init__(self, align: int = 64) -> None:
    """Loads a saved model.

    Args:
      align: 'If >1, pad the input size so it divides with this before
        inference.'
    """
    self._model = hub.load("/kaggle/input/film/tensorflow2/film/1")
    self._align = align

  def __call__(self, x0: np.ndarray, x1: np.ndarray,
               dt: np.ndarray) -> np.ndarray:
    """Generates an interpolated frame between given two batches of frames.

    All inputs should be np.float32 datatype.

    Args:
      x0: First image batch. Dimensions: (batch_size, height, width, channels)
      x1: Second image batch. Dimensions: (batch_size, height, width, channels)
      dt: Sub-frame time. Range [0,1]. Dimensions: (batch_size,)

    Returns:
      The result with dimensions (batch_size, height, width, channels).
    """
    if self._align is not None:
      x0, bbox_to_crop = _pad_to_align(x0, self._align)
      x1, _ = _pad_to_align(x1, self._align)

    inputs = {'x0': x0, 'x1': x1, 'time': dt[..., np.newaxis]}
    result = self._model(inputs, training=False)
    image = result['image']

    if self._align is not None:
      image = tf.image.crop_to_bounding_box(image, **bbox_to_crop)
    return image.numpy()

## Frame and Video Generation Utility Functions

In [ ]:
def _recursive_generator(
    frame1: np.ndarray, frame2: np.ndarray, num_recursions: int,
    interpolator: Interpolator) -> Generator[np.ndarray, None, None]:
  """Splits halfway to repeatedly generate more frames.

  Args:
    frame1: Input image 1.
    frame2: Input image 2.
    num_recursions: How many times to interpolate the consecutive image pairs.
    interpolator: The frame interpolator instance.

  Yields:
    The interpolated frames, including the first frame (frame1), but excluding
    the final frame2.
  """
  if num_recursions == 0:
    yield frame1
  else:
    # Adds the batch dimension to all inputs before calling the interpolator,
    # and remove it afterwards.
    time = np.full(shape=(1,), fill_value=0.5, dtype=np.float32)
    mid_frame = interpolator(
        np.expand_dims(frame1, axis=0), np.expand_dims(frame2, axis=0), time)[0]
    yield from _recursive_generator(frame1, mid_frame, num_recursions - 1,
                                    interpolator)
    yield from _recursive_generator(mid_frame, frame2, num_recursions - 1,
                                    interpolator)


def interpolate_recursively(
    frames: List[np.ndarray], num_recursions: int,
    interpolator: Interpolator) -> Iterable[np.ndarray]:
  """Generates interpolated frames by repeatedly interpolating the midpoint.

  Args:
    frames: List of input frames. Expected shape (H, W, 3). The colors should be
      in the range[0, 1] and in gamma space.
    num_recursions: Number of times to do recursive midpoint
      interpolation.
    interpolator: The frame interpolation model to use.

  Yields:
    The interpolated frames (including the inputs).
  """
  n = len(frames)
  for i in range(1, n):
    yield from _recursive_generator(frames[i - 1], frames[i],
                                    times_to_interpolate, interpolator)
  # Separately yield the final frame.
  yield frames[-1]

In [ ]:
times_to_interpolate = 6
interpolator = Interpolator()

## Running the Interpolator

In [ ]:
input_frames = [image1, image2]
frames = list(
    interpolate_recursively(input_frames, times_to_interpolate,
                                        interpolator))


In [ ]:
print(f'video with {len(frames)} frames')
media.show_video(frames, fps=30, title='FILM interpolated video')

For more information, you can visit [FILM's model repository](https://github.com/google-research/frame-interpolation).


## Citation

If you find this model and code useful in your works, please acknowledge it appropriately by citing:

```
@inproceedings{reda2022film,
 title = {FILM: Frame Interpolation for Large Motion},
 author = {Fitsum Reda and Janne Kontkanen and Eric Tabellion and Deqing Sun and Caroline Pantofaru and Brian Curless},
 booktitle = {The European Conference on Computer Vision (ECCV)},
 year = {2022}
}
```

```
@misc{film-tf,
  title = {Tensorflow 2 Implementation of "FILM: Frame Interpolation for Large Motion"},
  author = {Fitsum Reda and Janne Kontkanen and Eric Tabellion and Deqing Sun and Caroline Pantofaru and Brian Curless},
  year = {2022},
  publisher = {GitHub},
  journal = {GitHub repository},
  howpublished = {\url{https://github.com/google-research/frame-interpolation}}
}
```

Credit: https://github.com/tensorflow/hub/blob/master/examples/colab/tf_hub_film_example.ipynb